- [Linear Regression](#linear-regression)
- [Logistic Regression + MCMC + Moment Matching](#Logistic-Regression-+-MCMC-+-Moment-Matching)
- [Logistic Regression + MCMC](#Logistic-Regression-+-MCMC)
- [Logistic Regression + PP Posterior Update](#Logistic-Regression-+-PP-Posterior-Update)
- [Online IRT](#Online-IRT)
- [Batch IRT](#Batch-IRT)

In [2]:
import probpy as pp
import numpy as np
import numba
import random

linear regression
---

In [2]:
%%time
def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]

w = [5, -2, 1, 5] # True underlying model
x = np.random.rand(100, 3) * 10
y = predict(w, x) + pp.normal.sample(mu=0, sigma=1, size=100).reshape(-1)



prior = pp.multivariate_normal.med(mu=np.ones(4) * 0, sigma=np.eye(4) * 5)
likelihood = pp.unilinear.med(sigma=1) # There exist an implementation for linear because it has a conjugate prior

for i in range(100):
    data = (y[i], x[i])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, priors=prior)
    
    if i % 10 == 0:
        w_approx = pp.mode(prior)
        print("Parameter Estimate", w_approx)
        
        print("Prior MSE", np.square(y - predict(w_approx, x)).mean(), 
              "True MSE", np.square(y - predict(w, x)).mean())
        print()

Parameter Estimate [0.21281153 0.59059513 0.57440103 0.06327144]
Prior MSE 655.8381332217971 True MSE 1.10134013589333

Parameter Estimate [ 4.88672306 -1.9801504   1.04075558  5.00902414]
Prior MSE 1.132116345911245 True MSE 1.10134013589333

Parameter Estimate [ 5.04784598 -1.93289676  0.91881914  4.57239463]
Prior MSE 1.2519989179279705 True MSE 1.10134013589333

Parameter Estimate [ 5.01613718 -1.96074208  1.03265169  4.45406485]
Prior MSE 1.1387997571378152 True MSE 1.10134013589333

Parameter Estimate [ 4.9812195  -2.00023052  0.99075426  5.13859662]
Prior MSE 1.0798460022121477 True MSE 1.10134013589333

Parameter Estimate [ 4.94284131 -2.04735263  1.02265148  5.34292567]
Prior MSE 1.0800441135412566 True MSE 1.10134013589333

Parameter Estimate [ 4.94586207 -2.00797347  1.0475599   4.99168838]
Prior MSE 1.0831393404030327 True MSE 1.10134013589333

Parameter Estimate [ 4.92608929 -1.97780335  1.01218862  5.17157878]
Prior MSE 1.0622888537359934 True MSE 1.10134013589333

Parame

Logistic Regression + MCMC + Moment Matching
---

In [3]:
%%time
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior

def likelihood(y, x, w):
    return pp.normal.p((y - sigmoid(x @ w[:, :-1, None] + w[:, None, None, -1]).squeeze(axis=2)),
                    mu=0.0, sigma=1.0)
    
    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 5)

for i in range(50):
    j = random.randint(0, 80)
    data = (y[j: j + 20], x[j: j + 20])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   priors=prior, 
                                   match_moments_for=pp.multivariate_normal,
                                   batch=30,
                                   samples=3000,
                                   mixing=200, 
                                   energies=1.0,
                                   mode="mcmc")

    if i % 10 == 0:
        w_approx = pp.mode(prior)
        print("Parameter Estimate", w_approx)
        
        print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
              "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
        print()

Parameter Estimate [-0.48975965  0.14963146  0.2350226  -1.00685625]
Prior MSE 0.11987487531336732 True MSE 0.02128228460748603

Parameter Estimate [-1.71138875  2.19177311  4.03296924 -2.69885113]
Prior MSE 0.022021863707658462 True MSE 0.02128228460748603

Parameter Estimate [-2.33872715  2.027784    4.48877893 -2.61688593]
Prior MSE 0.02133894995512444 True MSE 0.02128228460748603

Parameter Estimate [-2.7700824   2.39768265  5.14620939 -2.90258804]
Prior MSE 0.020853957575349885 True MSE 0.02128228460748603

Parameter Estimate [-3.06909024  2.80777047  5.49375224 -3.09583722]
Prior MSE 0.021046664348029345 True MSE 0.02128228460748603

CPU times: user 19.5 s, sys: 29.4 s, total: 48.9 s
Wall time: 13.1 s


Logistic Regression + MCMC
---

In [4]:
%%time
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior

def likelihood(y, x, w):
    return pp.normal.p((y - sigmoid(x @ w[:, :-1, None] + w[:, None, None, -1]).squeeze(axis=2)),
                    mu=0.0, sigma=1.0)
    
    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 5)

for i in range(50):
    j = random.randint(0, 80)
    data = (y[j: j + 20], x[j: j + 20])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   priors=prior, 
                                   batch=5,
                                   samples=1000,
                                   mixing=100, 
                                   energies=1.0,
                                   mode="mcmc")

    if i % 10 == 0:
        modes = pp.mode(prior) # modes are sorted in order first is largest

        print("Number of modes", len(modes))
        w_approx = modes[0]
        
        print("Parameter Estimate", w_approx)
        
        print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
              "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
        print()

Number of modes 3
Parameter Estimate [-0.94819854  1.66953208  0.196584   -0.86041563]
Prior MSE 0.0687937816364164 True MSE 0.02928557812574474

Number of modes 1
Parameter Estimate [-2.20070917  1.67368058  3.28076658 -2.24555451]
Prior MSE 0.0371549385585871 True MSE 0.02928557812574474

Number of modes 1
Parameter Estimate [-2.88225696  2.7658537   4.28918435 -2.90840339]
Prior MSE 0.03134425384020127 True MSE 0.02928557812574474

Number of modes 4
Parameter Estimate [-2.94845944  3.77909141  4.72005859 -3.58584948]
Prior MSE 0.03293918569492301 True MSE 0.02928557812574474

Number of modes 5
Parameter Estimate [-2.4104538   4.67787955  5.8707972  -4.93223359]
Prior MSE 0.039656936972431625 True MSE 0.02928557812574474

CPU times: user 27.3 s, sys: 1.59 s, total: 28.9 s
Wall time: 27.4 s


Logistic Regression + PP Posterior Update
---


In [3]:
%%time
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior
def likelihood(y, x, w):
    return pp.normal.p((y - sigmoid(x @ w[:, :-1, None] + w[:, None, None, -1]).squeeze(axis=2)),
                    mu=0.0, sigma=1.0)
    
    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 10)

for i in range(5):
    data = (y, x)
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   priors=prior, 
                                   batch=500,
                                   samples=50000,
                                   mixing=10000, 
                                   energies=1.0,
                                   mode="ga",
                                   use_cl=True, # Set this to false to make much faster (but poorer posterior estimate)
                                   learning_rate=1.0,
                                   cl_iterations=10,
                                   verbose=False)

    modes = pp.mode(prior) # modes are sorted in order first is largest

    
    print("Number of modes", len(modes))
    for mode in modes:
        print(mode)

    w_approx = modes[0]

    print("Parameter Estimate", w_approx)

    print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
          "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
    print()

Number of modes 5
[-2.78790888  1.27165004  2.84500899 -1.0959795 ]
[-2.3667205   2.23198221  4.1314479  -2.58905247]
[-0.45641238  1.25587952  2.98018055 -1.96978248]
[-2.0288135   2.72378847  2.40384992 -1.95928287]
[-1.26419155  2.05771503  2.76033245 -2.11435715]
Parameter Estimate [-2.78790888  1.27165004  2.84500899 -1.0959795 ]
Prior MSE 0.032197082744360575 True MSE 0.019077171723090387

Number of modes 4
[-2.23734495  2.07930446  4.42862383 -2.56226562]
[-2.10446086  2.19867819  3.73731272 -2.30461506]
[-2.47568095  1.23771093  2.5690527  -1.20355688]
[-0.89791507  1.47889424  2.66148943 -2.25940566]
Parameter Estimate [-2.23734495  2.07930446  4.42862383 -2.56226562]
Prior MSE 0.022075573411077604 True MSE 0.019077171723090387

Number of modes 1
[-2.27496647  2.21226172  3.96144206 -2.53397745]
Parameter Estimate [-2.27496647  2.21226172  3.96144206 -2.53397745]
Prior MSE 0.021911859792054428 True MSE 0.019077171723090387

Number of modes 2
[-2.32267933  2.51427628  4.0544283

Online-IRT
---

In [10]:
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def logit(x):
    return np.log(x / (1 - x))

student_skill = logit(0.7)

items = logit(np.array([0.4, 0.6, 0.8, 0.7]))  # difficulties

def likelihood(obs, item, skill):
    result = []
    for _skill in skill:
        result.append(pp.normal.p(obs - sigmoid(_skill - item), mu=0.0, sigma=0.6))

    return np.array(result)

samples = 30
obs, its = [], []
for i in range(samples):  # 100 samples
    item = items[np.random.randint(0, items.size)]
    outcome = (np.random.rand() < sigmoid(student_skill - item)).astype(np.float)

    obs.append(outcome)
    its.append(item)

prior_skill = pp.normal.med(mu=0.0, sigma=10)

for i in range(samples):
    prior_skill = pp.parameter_posterior((obs[i], its[i]), likelihood=likelihood, priors=prior_skill,
                                         mode="mcmc", match_moments_for=pp.normal,
                                         samples=20000, mixing=3000, batch=5)
    modes = sigmoid(np.array(pp.mode(prior_skill)))
    
    print(modes)



0.6491485501404993
0.7933167902495826
0.8599585939752366
0.8744040533216224
0.8863141581793198
0.9171030022338412
0.8738617001458413
0.8981502419096364
0.914618461423574
0.8871862788534326
0.842540573662585
0.7946864358368195
0.7376521108765716
0.7058602615270904
0.71563868405117
0.7663080878768663
0.7357893070207435
0.706783047710064
0.7192188655584557
0.750822654713278
0.7227023123786681
0.7026900941471164
0.7179915519887132
0.7051534734789741
0.6866450777399479
0.6967853938220204
0.6766761673222068
0.6899540605656267
0.7038988645530234
0.7115519734586352


Batch IRT
---

In [ ]:
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def logit(x):
    return np.log(x / (1 - x))

student_skill = logit(0.7)

items = logit(np.array([0.4, 0.6, 0.8, 0.7]))  # difficulties

def likelihood(obs, item, skill):
    result = []
    for _skill in skill:
        result.append(pp.normal.p(obs - sigmoid(_skill - item), mu=0.0, sigma=0.6))

    return np.array(result)

samples = 100
obs, its = [], []
for i in range(samples):  # 100 samples
    item = items[np.random.randint(0, items.size)]
    outcome = (np.random.rand() < sigmoid(student_skill - item)).astype(np.float)

    obs.append(outcome)
    its.append(item)

prior_skill = pp.normal.med(mu=0.0, sigma=10)

for i in range(3):
    prior_skill = pp.parameter_posterior((obs, its), likelihood=likelihood, priors=prior_skill,
                                         mode="ga", samples=30000, mixing=0, batch=500,
                                         energies=5.0, bases=1, use_cl=True, variance=3)
    modes = sigmoid(np.array(pp.mode(prior_skill)))
    
    print(modes)


[[0.69557953]]
[[0.69506556]
 [0.68340836]]
